In [ ]:
pip install transformers torch sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 48.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [1]:
from transformers import BartForConditionalGeneration, BartTokenizer
import nltk
from nltk.tokenize import sent_tokenize

# Télécharger les données nécessaires pour NLTK (pour la tokenization des phrases)
nltk.download('punkt')

class AbstractiveSummarizer:
    def __init__(self, model_name="facebook/bart-large-cnn"):
        """
        Initialise le résumeur avec le modèle BART pré-entraîné
        :param model_name: nom du modèle pré-entraîné (par défaut facebook/bart-large-cnn)
        """
        self.model = BartForConditionalGeneration.from_pretrained(model_name)
        self.tokenizer = BartTokenizer.from_pretrained(model_name)
        self.max_input_length = 1024  # limite de tokens pour BART

    def chunk_text(self, text, max_length=1000):
        """
        Découpe le texte en chunks si trop long pour le modèle
        :param text: texte à découper
        :param max_length: longueur maximale d'un chunk
        :return: liste de chunks de texte
        """
        sentences = sent_tokenize(text)
        chunks = []
        current_chunk = ""

        for sentence in sentences:
            if len(current_chunk) + len(sentence) < max_length:
                current_chunk += " " + sentence
            else:
                chunks.append(current_chunk.strip())
                current_chunk = sentence

        if current_chunk:
            chunks.append(current_chunk.strip())

        return chunks

    def summarize(self, text, max_length=150, min_length=40, num_beams=4, length_penalty=2.0):
        """
        Génère un résumé abstractif du texte
        :param text: texte à résumer
        :param max_length: longueur maximale du résumé
        :param min_length: longueur minimale du résumé
        :param num_beams: nombre de beams pour la recherche
        :param length_penalty: pénalité de longueur (>1 encourage les résumés plus longs)
        :return: résumé généré
        """
        # Découper le texte si nécessaire
        if len(self.tokenizer(text)['input_ids']) > self.max_input_length:
            chunks = self.chunk_text(text)
            summaries = []

            for chunk in chunks:
                inputs = self.tokenizer([chunk], max_length=self.max_input_length, truncation=True, return_tensors="pt")
                summary_ids = self.model.generate(
                    inputs['input_ids'],
                    num_beams=num_beams,
                    max_length=max_length,
                    min_length=min_length,
                    length_penalty=length_penalty,
                    early_stopping=True
                )
                summaries.append(self.tokenizer.decode(summary_ids[0], skip_special_tokens=True))

            # Combiner les résumés partiels et résumer à nouveau
            combined_summary = " ".join(summaries)
            return self.summarize(combined_summary, max_length, min_length, num_beams, length_penalty)
        else:
            inputs = self.tokenizer([text], max_length=self.max_input_length, truncation=True, return_tensors="pt")
            summary_ids = self.model.generate(
                inputs['input_ids'],
                num_beams=num_beams,
                max_length=max_length,
                min_length=min_length,
                length_penalty=length_penalty,
                early_stopping=True
            )
            return self.tokenizer.decode(summary_ids[0], skip_special_tokens=True)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Exemple d'utilisation
if __name__ == "__main__":
    summarizer = AbstractiveSummarizer()

    # Texte à résumer (vous pouvez remplacer par n'importe quel texte)
    text = """
    Le réchauffement climatique est un phénomène global de transformation du climat caractérisé
    par une augmentation générale des températures moyennes, et qui modifie durablement les
    équilibres météorologiques et les écosystèmes. Il est principalement causé par les émissions
    de gaz à effet de serre d'origine humaine, qui augmentent l'effet de serre. Les conséquences
    du réchauffement climatique sont multiples : fonte des glaciers, montée des eaux, augmentation
    de la fréquence des événements météorologiques extrêmes, perturbations des écosystèmes et
    menaces sur la biodiversité. Pour lutter contre ce phénomène, les États ont adopté l'accord
    de Paris en 2015, visant à limiter le réchauffement bien en dessous de 2°C par rapport aux
    niveaux préindustriels. Cela nécessite une transition énergétique vers des sources d'énergie
    renouvelables et une réduction drastique des émissions de CO2.
    """

    summary = summarizer.summarize(text)
    print("Résumé abstractif généré:")
    print(summary)

Résumé abstractif généré:
Le réchauffement climatique modifie durablement les équilibres météorologiques and les écosystèmes. Il est principalement causé by les émissions    de gaz à effet de serre d'origine humaine. Les États ont adopté l'accord    de Paris en 2015, visant à limiter le réchAUffement.
